<a href="https://colab.research.google.com/github/pdrobny/Potential_Talents/blob/main/HF_gemma_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HugginFace with Gemma after fine-tuning






## Setup

In [1]:
!pip install transformers sentence-transformers
!pip install transformers torch
!pip install -U bitsandbytes
!pip install datasets
!pip install accelerate
!pip install peft
!pip install -U trl

In [2]:
# import libraries
import pandas as pd
import numpy as np
import warnings
import logging
import random
import requests
import sys
import torch
import peft
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM, LlamaTokenizer, set_seed, TrainingArguments
from huggingface_hub import notebook_login
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from trl import SFTTrainer
from datasets import Dataset
warnings.filterwarnings('ignore', category=UserWarning)

print(torch.__version__)
#tf.__version__

2.8.0+cu126


# Data prep

In [3]:
#from google.colab import drive#files
#uploaded = files.upload()
#drive.mount('/content/drive/My Drive/Colab Notebooks')
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir('/content/drive/My Drive/Colab Notebooks/')

df = pd.read_csv('talents.csv')
df.head(20)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


,id,title,sentence_bert_cossim
0,1,innovative and driven professional seeking a r...,1.000000
1,431,aspiring data science professional focused on ...,0.769162
2,544,data analyst data scientist business analyst d...,0.768222
3,833,data analyst turning complex data into actiona...,0.745245
4,199,ms in information systems northeastern univers...,0.727268
5,28,aspiring data scientist passion for data-drive...,0.720545
6,1282,data scientist and analyst driving business in...,0.717432
7,426,master of science in analytics at georgia inst...,0.717093
8,963,passionate data scientist seeking exciting opp...,0.711745
9,487,research assistant penn state seeking opportun...,0.708959


In [4]:
job_titles = df["title"].tolist()


In [5]:
job_titles_short = df["title"].head(10).tolist()
job_titles_short

['innovative and driven professional seeking a role in data analyticsdata science in the information technology industry.',
 'aspiring data science professional focused on data analysis machine learning and data visualization actively seeking opportunities',
 'data analyst data scientist business analyst driving data-driven insights strategic solutions',
 'data analyst turning complex data into actionable insights passionate about solving business challenges with data-driven solutions',
 'ms in information systems northeastern university data scientist business intelligence data engineering data analyst transforming data into insights',
 'aspiring data scientist passion for data-driven decision making master of science in business analytics graduate  university of new hampshire',
 'data scientist and analyst driving business insights with advanced data techniques research expertise',
 'master of science in analytics at georgia institute of technology aspiring data scientist',
 'passion

In [6]:
job_ids_short = df["id"].head(10).tolist()
job_ids_short

[1, 431, 544, 833, 199, 28, 1282, 426, 963, 487]

In [7]:
target_title = "data analyst"

In [8]:
notebook_login()

## Fine-Tuning Model

In [9]:
#!pip install accelerate

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer
##from accelerate import disk_offload

model_id = "google/gemma-1.1-2b-it"  # Update based on your available resources
##model_id = "google/gemma-2-2b-it" # example
#tokenizer = AutoTokenizer.from_pretrained(model_id)
#model = AutoModelForCausalLM.from_pretrained(
#    model_id,
#    load_in_4bit=True,
#    device_map="auto"
#)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
model_id,
torch_dtype=None, # let HF choose; avoids mixups
device_map="auto"#cpu
)

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# QLoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

data = [
    {
        "prompt": f"""
Return a list of the top 5 job candidates with full job title and job id from a job titles list ranked by their similirality to the search term in desecnding order.  Only show the answer. Do not reason or explain.
**Search term**
{target_title}

**job titles**
{job_titles_short}

**job ids**
{job_ids_short}


Show answer in following format:
Rank Job ID   Job Title
1 - 1: innovative and driven professional seeking a role in data analyticsdata science in the information technology industry.
2 - ...
3 - ...
...
Answer: Top 5 are:
""",
        "response": """Rank Job ID   Job Title
1 - 199: data analyst turning complex data into actionable insights passionate about solving business challenges with data-driven solutions
2 - 431: aspiring data science professional focused on data analysis machine learning and data visualization actively seeking opportunities
3 - 833: data scientist and analyst driving business insights with advanced data techniques research expertise
4 - 28: data analyst data scientist business analyst driving data-driven insights strategic solutions
5 - 426: ms in information systems northeastern university data scientist business intelligence data engineering data analyst transforming data into insights"""
    }
]
# Convert to Hugging Face Dataset format
dataset = Dataset.from_list([
    {"text": f"{item['prompt']}{item['response']}"} for item in data
])

training_args = TrainingArguments(
output_dir="out",
per_device_train_batch_size=1,
gradient_accumulation_steps=4,
learning_rate=2e-5,
num_train_epochs=1,
fp16=False, # start safe
bf16=False,
logging_steps=10,
save_steps=1000,
report_to="none"
)

#peft_config = LoraConfig(
#    r=8,
#    lora_alpha=16,
#    target_modules=["query_key_value"],
#    lora_dropout=0.05,
#    bias="none",
#    task_type="CAUSAL_LM"
#)

trainer = SFTTrainer(
model=model,

train_dataset=dataset, # your prepared dataset
args=training_args,
)
trainer.train()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Adding EOS to train dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Step,Training Loss


TrainOutput(global_step=1, training_loss=3.2088983058929443, metrics={'train_runtime': 3.5085, 'train_samples_per_second': 0.285, 'train_steps_per_second': 0.285, 'total_flos': 5900830703616.0, 'train_loss': 3.2088983058929443, 'entropy': 0.8526465892791748, 'num_tokens': 496.0, 'mean_token_accuracy': 0.5313131213188171, 'epoch': 1.0})

In [18]:
#merged_model_dir = "./gemma-cpu-finetune-merged"

# Save the fine-tuned model
#trainer.model.save_pretrained(merged_model_dir)

# Save the tokenizer to the same directory
#tokenizer.save_pretrained(merged_model_dir)

# Load the merged model and tokenizer explicitly
#merged_tokenizer = AutoTokenizer.from_pretrained(merged_model_dir)
#merged_model = AutoModelForCausalLM.from_pretrained(merged_model_dir)




In [11]:
##seed = random.randint(1000,9999)
#seed = 7308
##set_seed(seed)
##print(seed)
#good seeds: 7308

# Create a combined string of job ID and job title pairs
job_pairs = "\n".join([f"{job_id}: {job_title}" for job_id, job_title in zip(job_ids_short, job_titles_short)])

# Load the merged model and tokenizer using the pipeline
# You might need to specify the trust_remote_code=True for some models
# Pass the loaded model and tokenizer objects to the pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)#, device='auto')


prompt = f"""
Return a list of the top 5 job candidates with full unmodified job title and matching job id from a job titles list ranked by their similirarity to the search term in desecnding order.  Only show the answer. Do not reason or explain.
**Search term**
{target_title}

**Job candidates (ID: Title)**
{job_pairs}


Show answer in following format:
Rank Job ID   Job Title
1 - 1: innovative and driven professional seeking a role in data analyticsdata science in the information technology industry.
2 - ...
3 - ...
...
Answer: Top 5 are:
"""

output = generator(prompt, max_new_tokens=200, num_return_sequences=1)#, device='cpu')
print(output[0]['generated_text'])

Device set to use cuda:0



Return a list of the top 5 job candidates with full unmodified job title and matching job id from a job titles list ranked by their similirarity to the search term in desecnding order.  Only show the answer. Do not reason or explain.
**Search term**
data analyst

**Job candidates (ID: Title)**
1: innovative and driven professional seeking a role in data analyticsdata science in the information technology industry.
431: aspiring data science professional focused on data analysis machine learning and data visualization actively seeking opportunities
544: data analyst data scientist business analyst driving data-driven insights strategic solutions
833: data analyst turning complex data into actionable insights passionate about solving business challenges with data-driven solutions
199: ms in information systems northeastern university data scientist business intelligence data engineering data analyst transforming data into insights
28: aspiring data scientist passion for data-driven deci

In [12]:
generated_text = output[0]['generated_text']
answer_start_index = generated_text.find("Answer:")
# Extract the answer and print it
if answer_start_index != -1:
    # Add the length of the search string to get the true start of the answer
    answer_start_index += len("Answer:")
    print(generated_text[answer_start_index:].strip())
else:
    # If the marker was not found, print the whole output or a message
    print("Could not find the start of the answer in the generated text.")
    print(generated_text)

Top 5 are:
1 - 199 - MS in Information Systems
2 - 487 - Research Assistant
3 - 544 - Data Analyst
4 - 833 - Data Analyst
5 - 28 - Aspiring Data Scientist


#Conclusion
The project 3 deals with finding candidates that fit into categories and finding how similar they are.

Term Frequency-Inverse Document Frequency is what TF-IDF means and is what I started with.
TF-IDF finds the frequency or amount of times that the word exists in a document given the number of words in a document, and this is the word's importance. The search term was used to find the cosine similarity between the search term and corpus. The answer showed how similar the search term was between different parts of the corpus.

I needed to find the word embeddings or a word's numerical representations so I utilized the three following libraries: Word2Vec, GloVe, and FastText. I used the search term "aspiring human resources" for the word embeddings libraries and compared it to the corpus.

Google's Word2Vec was the first library I used. A word's relationships are compared with word2vec. Words used in a similar context to the same word in it's context have similar meanings. The neighboring words build the word embeddings for every word in a document or corpus. The Word2Vec gave the cosine similarity values of the search term to the corpus and listed them in descending order.

Stanford University's GloVe was the next library I used. GloVe stands for Global Vectors for Word Representation. GloVe builds a co-occurrence matrix for each word with the goal of comparing how often words appear with one another in a document or corpus. The results of the GloVe were similar to Word2Vec as they were in descending order, but the cosine similarity values were different.

Facebook's FastText was the last library I used. FastText breaks up words into smaller n-grams or subwords. The result of this for a word the n-gram's sum or average is the final vector. The results of the FastText were similar to Word2Vec and GloVe as they were in descending order, but the cosine similarity values were different.

BERT stands for Bidirectional Encoder Representations from Transformers. BERT is a sentence transformer. BERT was used with cosine similarity to find contextual embeddings which shows their semantic similarity. BERT reads both from left to right and from right to left simultaneously. The entire sentence is used with BERT and it also give a cosine similarity score like previous libraries.

PyTorch from Facebook is a machine learning framework used in natural language processing. PyTorch was utilized to find the embeddings score along with the old and new scores.

File 2 used PyTorch, which was used with the SmolLM2, Alibaba's Qwen, and Meta's Llama models from HuggingFace. The models are large language models (LLMs) and are generative AI models. I asked the questions to the models and they thought about what to say, giving their best answer. Next I loaded my job titles list and search term and the model thought of the best answers and how similar they were.

In File 3 the last task, which was to perform was fine-tuning. Fine-tuning works by choosing a task that is specific and training it on a smaller dataset. Google's Gemma was the model used for fine-tuning. My model found job titles similar to the term data science and listed the top 5.